# Post-wildfire vegetation regrowth analysis
Author: Stepan Bryleev

### Moisture Stress Index (MSI) 

In this Jupyter Notebook there is a JavaScript code for calculating and displaying Moisture Stress Index (MSI) values for the study area in GEE using Landsat Surface Reflectance data.

* Moisture Stress Index is used for canopy stress analysis and productivity prediction.
* $MSI=\frac{MidIR}{NIR}$

```js
// ==============================================================  //
// This part of the tutorial is for creating Landsat GRSM MSI maps //
// ==============================================================  //

// Specify center location and for GRSM
var SiteCenterPoint = ee.Geometry.Point([-83.5, 35.7]);

// Center the map on our area of interest and set zoom level
Map.setCenter(-83.5, 35.7, 12);

var study_polygon = /* color: #98ff00 */ee.Geometry.Polygon(
        [[[-83.5425714556907, 35.701602862979236],
          [-83.5422281329368, 35.690310616971246],
          [-83.53879490539774, 35.63828989061714],
          [-83.5037759844993, 35.615406781430224],
          [-83.48592430887022, 35.61938453875363],
          [-83.47459355041727, 35.627546807436595],
          [-83.47922808124717, 35.65117980654369],
          [-83.47738302453864, 35.682051678012314],
          [-83.4774257678571, 35.69326203209512],
          [-83.47751235961914, 35.71456632200802],
          [-83.47837061486136, 35.7182597023649],
          [-83.48043060302734, 35.72055934657099],
          [-83.53656387329102, 35.70494866628485]]]);

// Define the GRSM perim variable
var grsm_boundary = ee.FeatureCollection('projects/ee-stbr4432/assets/grsm_polygon');

// Define the fire perimeter variable
var fire_perimeter = ee.FeatureCollection('projects/ee-stbr4432/assets/chimney_tops_perim');
var fireBoundGeom = fire_perimeter.geometry();

// Apply the intersection method to the Polygon object.
var polygonIntersection = fireBoundGeom.intersection(grsm_boundary);
print('polygon intersection',polygonIntersection);

// Create study area.
var studyArea = study_polygon.intersection(polygonIntersection);

// Print polygon area in square kilometers.
print('Study area square: ', studyArea.area().divide(1000 * 1000));

// Print polygon perimeter length in kilometers.
print('Study area perimeter: ', studyArea.perimeter().divide(1000));


// ===== Create a cloud masking function ===== //

// Apply cloud masking
function ls8_mask(image) {
  // Develop masks for unwanted pixels (fill, cloud, cloud shadow).
  // Bits 3 and 5 are cloud shadow and cloud, respectively.
  var cloudShadowBitMask = (1 << 3);
  var cloudsBitMask = (1 << 5);
  // Get the pixel QA band.
  var qa = image.select('QA_PIXEL');
  // Both flags should be set to zero, indicating clear conditions.
  var mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    .and(qa.bitwiseAnd(cloudsBitMask).eq(0));
  var saturationMask = image.select('QA_RADSAT').eq(0);

  // Apply the scaling factors to the appropriate bands.
  var getFactorImg = function(factorNames) {
    var factorList = image.toDictionary().select(factorNames).values();
    return ee.Image.constant(factorList);
  };
  var scaleImg = getFactorImg([
    'REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10']);
  var offsetImg = getFactorImg([
    'REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10']);
  var scaled = image.select('SR_B.|ST_B10').multiply(scaleImg).add(offsetImg);

  // Replace original bands with scaled bands and apply masks
  return image.addBands(scaled, null, true).updateMask(mask).updateMask(saturationMask);
}


// === Create a function to calculate MSI image with a date === //

// MSI = MidIR / NIR
// MSI (Landsat 8) = SR_B6 / SR_B5

// Define a function and bands of interest
function calc_msi(image, date) {
  // NIR = LS Band SR_B5, MidIR = LS Band SR_B6
  var ls_image_nir = image.select(["SR_B5"]);
  var ls_image_midir = image.select(["SR_B6"]);
  // Calculate MSI
  var ls_msi =  ls_image_midir.divide(ls_image_nir);
  
  // Display the result (optional)
  // var viz_params = {min:0, max:4, palette:['darkgreen', 'green', 'white', 'pink']};
  // Map.addLayer(ls_msi, viz_params, "GRSM MSI " + date, 0);
  
  return ls_msi.rename('MSI');
}


// // === Create a function to calculate dMSI image multipied by 1000 === //

// // Define a function
// function calc_dMSI(pre_fire_im, post_fire_im) {
//   // Substract images and multiply by 1000
//   var dMSI_unscaled = pre_fire_im.subtract(post_fire_im);
//   var dMSI_image = dMSI_unscaled.multiply(1000);

//   return dMSI_image;
// }


// === Create a function to plot histogram charts === //

// Define a function
function plot_hist(msi_image, studyArea, scale, title) {
  var msi_hist = ui.Chart.image.histogram({image: msi_image, region: studyArea, scale: scale})
      .setOptions({title: title,
            hAxis: {title: 'MSI Value',titleTextStyle: {italic: false, bold: true},},
            vAxis: {title: 'Count', titleTextStyle: {italic: false, bold: true}},});
            
  print(msi_hist);
}


// DEFINE DATES OF INTEREST:

var date_2015 = '2015-06-26';
var date_2016 = '2016-07-14';
var date_2017 = '2017-05-14'; 
var date_2018 = '2018-07-04';
var date_2019 = '2019-06-21';
var date_2020 = '2020-06-07';
var date_2021 = '2021-06-26';
var date_2022 = '2022-06-06';
var date_2023 = '2023-06-09';


// ======= Read 2015 Landsat data ======= //

// Read in the 2015 LS Image at GRSM and clip to Study Area
var LS_image_2015 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_019035_20150626").clip(studyArea);
// Apply masking function to 2015 image
var LS_image_2015_masked = ls8_mask(LS_image_2015);


// ======= Read 2016 Landsat data ======= //

// Read in the 2016 LS Image at GRSM and clip to Study Area
var LS_image_2016 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_019035_20160714").clip(studyArea);
// Apply masking function to 2016 image
var LS_image_2016_masked = ls8_mask(LS_image_2016);


// ======= Read 2017 Landsat data ======= //

// Read in the 2017 LS Image at GRSM and clip to Study Area
var LS_image_2017 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_019035_20170514").clip(studyArea);
// Apply masking function to 2017 image
var LS_image_2017_masked = ls8_mask(LS_image_2017);


// ======= Read 2018 Landsat data ======= //

// Read in the 2018 LS Image at GRSM and clip to Study Area
var LS_image_2018 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_019035_20180704").clip(studyArea);
// Apply masking function to 2018 image
var LS_image_2018_masked = ls8_mask(LS_image_2018);


// ======= Read 2019 Landsat data ======= //

// Read in the 2019 LS Image at GRSM and clip to Study Area
var LS_image_2019 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_019035_20190621").clip(studyArea);
// Apply masking function to 2019 image
var LS_image_2019_masked = ls8_mask(LS_image_2019);


// ======= Read 2020 Landsat data ======= //

// Read in the 2020 LS Image at GRSM and clip to Study Area
var LS_image_2020 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_019035_20200607").clip(studyArea);
// Apply masking function to 2020 image
var LS_image_2020_masked = ls8_mask(LS_image_2020);


// ======= Read 2021 Landsat data ======= //

// Read in the 2021 LS Image at GRSM and clip to Study Area
var LS_image_2021 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_019035_20210626").clip(studyArea);
// Apply masking function to 2021 image
var LS_image_2021_masked = ls8_mask(LS_image_2021);


// ======= Read 2022 Landsat data ======= //

// Read in the 2022 LS Image at GRSM and clip to Study Area
var LS_image_2022 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_018035_20220606").clip(studyArea);
// Apply masking function to 2021 image
var LS_image_2022_masked = ls8_mask(LS_image_2022);


// ======= Read 2023 Landsat data ======= //

// Read in the 2023 LS Image at GRSM and clip to Study Area
var LS_image_2023 = ee.Image("LANDSAT/LC08/C02/T1_L2/LC08_018035_20230609").clip(studyArea);
// Apply masking function to 2021 image
var LS_image_2023_masked = ls8_mask(LS_image_2023);


// ======= Calculate and plot MSI for each date of interest ======= //

var pre_fire_msi_2015 = calc_msi(LS_image_2015_masked, date_2015);
var pre_fire_msi_2016 = calc_msi(LS_image_2016_masked, date_2016);
var post_fire_msi_2017 = calc_msi(LS_image_2017_masked, date_2017);
var post_fire_msi_2018 = calc_msi(LS_image_2018_masked, date_2018);
var post_fire_msi_2019 = calc_msi(LS_image_2019_masked, date_2019);
var post_fire_msi_2020 = calc_msi(LS_image_2020_masked, date_2020);
var post_fire_msi_2021 = calc_msi(LS_image_2021_masked, date_2021);
var post_fire_msi_2022 = calc_msi(LS_image_2022_masked, date_2022);
var post_fire_msi_2023 = calc_msi(LS_image_2023_masked, date_2023);


// ======= Create MSI Classification and add MSI images to the map ======= //

// SLD = Styled Layer Descriptor

// Define an SLD style of discrete intervals to apply to the image.
var sld_intervals =
  '<RasterSymbolizer>' +
    '<ColorMap type="intervals" extended="false" >' + 
      '<ColorMapEntry color="#006400" quantity="0.2" label="0.2" />' + // DarkGreen
      '<ColorMapEntry color="#228B22" quantity="0.4" label="0.4" />' + // ForestGreen
      '<ColorMapEntry color="#32CD32" quantity="0.6" label="0.6" />' + // LimetGreen
      '<ColorMapEntry color="#7FFFD4" quantity="0.8" label="0.8" />' + // Aquamarine
      '<ColorMapEntry color="#ADFF2F" quantity="1" label="1" />' + // GreenYellow
      '<ColorMapEntry color="#FFFFE0" quantity="1.2" label="1.2" />' + // LightYellow
      '<ColorMapEntry color="#FFD700" quantity="1.4" label="1.4" />' + // Gold
      '<ColorMapEntry color="#FFC0CB" quantity="1.6" label="1.6" />' + // Pink
      '<ColorMapEntry color="#FF69B4" quantity="1.8" label="1.8" />' + // HotPink
      '<ColorMapEntry color="#FF1493" quantity="2" label="2" />' + // DeepPink
      '<ColorMapEntry color="#C71585" quantity="2.5" label="2.5" />' + // MediumVioletRed
      '<ColorMapEntry color="#8B008B" quantity="6" label="6" />' + // DarkMagenta
    '</ColorMap>' +
  '</RasterSymbolizer>';

// Add MSI images to the map using the color classification
Map.addLayer(pre_fire_msi_2015.sldStyle(sld_intervals), {}, '2015 MSI classified');
Map.addLayer(pre_fire_msi_2016.sldStyle(sld_intervals), {}, '2016 MSI classified');
Map.addLayer(post_fire_msi_2017.sldStyle(sld_intervals), {}, '2017 MSI classified');
Map.addLayer(post_fire_msi_2018.sldStyle(sld_intervals), {}, '2018 MSI classified');
Map.addLayer(post_fire_msi_2019.sldStyle(sld_intervals), {}, '2019 MSI classified');
Map.addLayer(post_fire_msi_2020.sldStyle(sld_intervals), {}, '2020 MSI classified');
Map.addLayer(post_fire_msi_2021.sldStyle(sld_intervals), {}, '2021 MSI classified');
Map.addLayer(post_fire_msi_2022.sldStyle(sld_intervals), {}, '2022 MSI classified');
Map.addLayer(post_fire_msi_2023.sldStyle(sld_intervals), {}, '2023 MSI classified');


// =======   ADD the MSI LEGEND  ======== //

// Create legend panel and set its position
var legend = ui.Panel({
  style: {
    position: 'bottom-right',
    padding: '8px 20px'
  }});
 
// Create legend title
var legendTitle = ui.Label({
  value: 'MSI Values',
  style: {fontWeight: 'bold',
    fontSize: '18px',
    margin: '0 0 6px 0',
    padding: '0'
    }});
 
// Add the title to the panel
legend.add(legendTitle);
 
// Creates and styles 1 row of the legend.
var makeRow = function(color, name) {
 
      // Create the label that is actually the colored box.
      var colorBox = ui.Label({
        style: {
          backgroundColor: '#' + color,
          // Use padding to give the box height and width.
          padding: '8px',
          margin: '0 0 4px 0'
        }});
 
      // Create the label filled with the description text.
      var description = ui.Label({
        value: name,
        style: {margin: '0 0 4px 6px'}
      });
 
      // return the panel
      return ui.Panel({
        widgets: [colorBox, description],
        layout: ui.Panel.Layout.Flow('horizontal')
      })};
 
//  Palette with the colors
var palette =['006400', '228B22', '32CD32', '7FFFD4', 'ADFF2F', 'FFFFE0',
              'FFD700', 'FFC0CB', 'FF69B4', 'FF1493', 'C71585', '8B008B'];
 
// Name of the legend
var names = ['<0.2', '0.2 ÷ 0.4', '0.4 ÷ 0.6', '0.6 ÷ 0.8', '0.8 ÷ 1', '1 ÷ 1.2',
             '1.2 ÷ 1.4', '1.4 ÷ 1.6', '1.6 ÷ 1.8', '1.8 ÷ 2', '2 ÷ 2.5', '> 2.5'];
 
// Add color and and names to a legend panel
for (var i = 0; i < 12; i++) {
  legend.add(makeRow(palette[i], names[i]));
  }  
 
// Add legend to map (alternatively you can also print the legend to the console)
Map.add(legend);


// ======= Display Fire Boundary ======= //

// Display the fire boundary
Map.addLayer(fire_perimeter.style({width: 2,
                                  color: "red",
                                  fillColor: "#00000000"}),{},"Fire Boundary", 1);


// ======= Plot histogram charts ======= //

// Plot histogram charts for each MSI image in the console
plot_hist(pre_fire_msi_2015, studyArea, 50, 'MSI 2015');
plot_hist(pre_fire_msi_2016, studyArea, 50, 'MSI 2016');
plot_hist(post_fire_msi_2017, studyArea, 50, 'MSI 2017');
plot_hist(post_fire_msi_2018, studyArea, 50, 'MSI 2018');
plot_hist(post_fire_msi_2019, studyArea, 50, 'MSI 2019');
plot_hist(post_fire_msi_2020, studyArea, 50, 'MSI 2020');
plot_hist(post_fire_msi_2021, studyArea, 50, 'MSI 2021');
plot_hist(post_fire_msi_2022, studyArea, 50, 'MSI 2022');
plot_hist(post_fire_msi_2023, studyArea, 50, 'MSI 2023');

// Create a list for a time series chart
var list = ee.List([pre_fire_msi_2015.copyProperties(LS_image_2015_masked, ['system:time_start']),
                    pre_fire_msi_2016.copyProperties(LS_image_2016_masked, ['system:time_start']),
                    post_fire_msi_2017.copyProperties(LS_image_2017_masked, ['system:time_start']),
                    post_fire_msi_2018.copyProperties(LS_image_2018_masked, ['system:time_start']),
                    post_fire_msi_2019.copyProperties(LS_image_2019_masked, ['system:time_start']),
                    post_fire_msi_2020.copyProperties(LS_image_2020_masked, ['system:time_start']),
                    post_fire_msi_2021.copyProperties(LS_image_2021_masked, ['system:time_start']),
                    post_fire_msi_2022.copyProperties(LS_image_2022_masked, ['system:time_start']),
                    post_fire_msi_2023.copyProperties(LS_image_2023_masked, ['system:time_start'])]);

var time_series_msi = ee.FeatureCollection(list);

// Create a time series chart, with image, geometry & median reducer
var plotMSI = ui.Chart.image.seriesByRegion(time_series_msi, studyArea, ee.Reducer.mean(), 
              'MSI', 100, 'system:time_start') // band, scale, x-axis property
              .setChartType('LineChart').setOptions({
                title: 'Mean MSI for Selected Geometry',
                hAxis: {title: 'Date'},
                vAxis: {title: 'MSI'},
                legend: {position: "none"},
                lineWidth: 1,
                pointSize: 3
});

// Display the chart
print(plotMSI);
```